In [1]:
!unzip dd.zip

Archive:  dd.zip
   creating: dd/
  inflating: dd/15_10_s.jpg          
  inflating: dd/15_11_s.jpg          
  inflating: dd/15_12_s.jpg          
  inflating: dd/15_13_s.jpg          
  inflating: dd/15_14_s.jpg          
  inflating: dd/15_15_s.jpg          
  inflating: dd/15_16_s.jpg          
  inflating: dd/15_17_s.jpg          
  inflating: dd/15_18_s.jpg          
  inflating: dd/15_1_s.jpg           
  inflating: dd/15_20_s.jpg          
  inflating: dd/15_21_s.jpg          
  inflating: dd/15_22_s.jpg          
  inflating: dd/15_23_s.jpg          
  inflating: dd/15_24_s.jpg          
  inflating: dd/15_2_s.jpg           
  inflating: dd/15_3_s.jpg           
  inflating: dd/15_4_s.jpg           
  inflating: dd/15_5_s.jpg           
  inflating: dd/15_6_s.jpg           
  inflating: dd/15_7_s.jpg           
  inflating: dd/15_8_s.jpg           
  inflating: dd/15_9_s.jpg           
  inflating: dd/16_10_s.jpg          
  inflating: dd/16_11_s.jpg          
  inflating: dd/

In [2]:
!pip install fuzzy-c-means

     |████████████████████████████████| 10.9 MB 7.8 MB/s 


In [14]:
import cv2
import os 
import numpy as np
import matplotlib.pyplot as plt
import shutil
from skimage.transform import resize
from fcmeans import FCM

# after unzipping dd.zip, folder is formed named dd which contained all images
# chnage address for new folder and run the code
dir_images = "dd/"
imgs = os.listdir(dir_images)

In [15]:
#function to get lbp
def lbpfn(img):
  lbim=np.zeros_like(img)
  for i in range(0,126):  # 128-3+1=126
    for j in range(0,126):
      im=img[i:i+3, j:j+3]  # taing 3X3 patches of images
      c=im[1][1]
      im=im.T.flatten()  # flattern it
      d=[]
      for a in im:
        if(a<c):
          d.append(a/c)
        else:
          d.append(c/a)
      d=np.delete(np.round(np.array(d)),4)  # delete the center pixel value
      lo=np.where(d)[0]
      if(len(lo)>0):
        n=np.sum(2**lo)  # converting to decimal value
      else:
        n=0
      lbim[i+1,j+1]=n  # putting the pixel value as the decimal value got
      # break
  return lbim

In [16]:
def spp(lbimg):  # fucntion to get the spp from lbp image
  # taking slices and concatinating the histograms
  d1=[]
  for i in range(0,128,32):
    for j in range(0,128,32):
      c=lbimg[i:i+32, j:j+32]
      vlbimg = c.flatten()
      hist,bins=np.histogram(vlbimg,bins=25,range=(0,255))  # taking bins =25
      d1.append(hist)
  for i in range(0,128,64):
    for j in range(0,128,64):
      c=lbimg[i:i+64, j:j+64]
      vlbimg = c.flatten()
      hist,bins=np.histogram(vlbimg,bins=25,range=(0,255))
      d1.append(hist)
  vlbimg = lbimg.flatten()
  hist,bins=np.histogram(vlbimg,bins=25,range=(0,255))
  d1.append(hist)  
  return d1


In [17]:
k=int(input("Enter number of cluster "))
# k=4
arrayofsp=[]
for im in imgs:
  img = cv2.imread(dir_images+im)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img = resize(img, (128, 128))
  lbimg=lbpfn(img)
  sp=spp(lbimg)
  # flat it then make kmeans
  arrayofsp.append(np.reshape(sp,-1))
a=np.array(arrayofsp)

Enter number of cluster4


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [19]:
# creating cluster from the extracted features
fcm = FCM(n_clusters=k) 
fcm.fit(a)
fcm_centers = fcm.centers
fcm_labels = fcm.predict(a)

In [20]:
# code to create output folder
parent="folder/"
os.mkdir(parent)

In [21]:
for na in range(1,k+1):
  fna=parent+"Cluster"+str(na)
  os.mkdir(fna)

In [22]:
for i,label in enumerate(fcm_labels):
  imname=imgs[i]
  fromcp=dir_images+imname
  label=label+1
  tocp=parent+"Cluster"+str(label)+"/"
  shutil.copy(fromcp, tocp)


In [23]:
# zipping the folder created

In [24]:
!zip -r final.zip folder

updating: folder/ (stored 0%)
updating: folder/Cluster3/ (stored 0%)
updating: folder/Cluster1/ (stored 0%)
updating: folder/Cluster4/ (stored 0%)
updating: folder/Cluster4/15_22_s.jpg (deflated 1%)
updating: folder/Cluster4/15_20_s.jpg (deflated 1%)
updating: folder/Cluster4/16_30_s.jpg (deflated 1%)
updating: folder/Cluster4/15_1_s.jpg (deflated 1%)
updating: folder/Cluster2/ (stored 0%)
updating: folder/Cluster2/15_14_s.jpg (deflated 1%)
updating: folder/Cluster2/16_23_s.jpg (deflated 2%)
updating: folder/Cluster2/15_23_s.jpg (deflated 1%)
updating: folder/Cluster2/15_6_s.jpg (deflated 1%)
updating: folder/Cluster2/15_15_s.jpg (deflated 2%)
updating: folder/Cluster2/16_28_s.jpg (deflated 2%)
updating: folder/Cluster2/16_10_s.jpg (deflated 1%)
updating: folder/Cluster2/16_24_s.jpg (deflated 2%)
updating: folder/Cluster2/16_18_s.jpg (deflated 2%)
updating: folder/Cluster2/16_6_s.jpg (deflated 1%)
updating: folder/Cluster2/15_18_s.jpg (deflated 1%)
updating: folder/Cluster2/15_10_s.jpg